In [1]:
import pandas as pd
from tqdm import tqdm

from joblib import Parallel, delayed
import multiprocessing as mp
from multiprocessing import cpu_count
from sklearn.model_selection import StratifiedGroupKFold, KFold

import tensorflow as tf
cpu_count()

4

In [2]:
train_df = pd.read_csv('/kaggle/input/asl-signs/train.csv')

In [3]:
pd.read_parquet('/kaggle/input/asl-signs/train_landmark_files/26734/1000035562.parquet')

,frame,row_id,type,landmark_index,x,y,z
0,20,20-face-0,face,0,0.494400,0.380470,-0.030626
1,20,20-face-1,face,1,0.496017,0.350735,-0.057565
2,20,20-face-2,face,2,0.500818,0.359343,-0.030283
3,20,20-face-3,face,3,0.489788,0.321780,-0.040622
4,20,20-face-4,face,4,0.495304,0.341821,-0.061152
...,...,...,...,...,...,...,...
12484,42,42-right_hand-16,right_hand,16,0.001660,0.549574,-0.145409
12485,42,42-right_hand-17,right_hand,17,0.042694,0.693116,-0.085307
12486,42,42-right_hand-18,right_hand,18,0.006723,0.665044,-0.114017
12487,42,42-right_hand-19,right_hand,19,-0.014755,0.643799,-0.123488


In [4]:
pd.read_parquet('/kaggle/input/asl-signs/train_landmark_files/26734/1000035562.parquet')[:543].type.value_counts()

face          468
pose           33
left_hand      21
right_hand     21
Name: type, dtype: int64

In [5]:
pd.read_parquet('/kaggle/input/asl-signs/train_landmark_files/26734/1000035562.parquet')[543:543*2].type.value_counts()

face          468
pose           33
left_hand      21
right_hand     21
Name: type, dtype: int64

In [6]:
import numpy as np
ROWS_PER_FRAME = 543
def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

In [7]:
import json
with open('/kaggle/input/asl-signs/sign_to_prediction_index_map.json') as json_file:
    LABEL_DICT = json.load(json_file)

In [8]:
def encode_row(row):
    coordinates = load_relevant_data_subset(f'/kaggle/input/asl-signs/{row.path}')
    coordinates_encoded = coordinates.tobytes()
    participant_id = int(row.participant_id)
    sequence_id = int(row.sequence_id)
    sign = int(LABEL_DICT[row.sign])
    record_bytes = tf.train.Example(features=tf.train.Features(feature={
                'coordinates': tf.train.Feature(bytes_list=tf.train.BytesList(value=[coordinates_encoded])),
                'participant_id': tf.train.Feature(int64_list=tf.train.Int64List(value=[participant_id])),
                'sequence_id':tf.train.Feature(int64_list=tf.train.Int64List(value=[sequence_id])),
                'sign':tf.train.Feature(int64_list=tf.train.Int64List(value=[sign])),
                })).SerializeToString()
    return record_bytes

def process_chunk(chunk, tfrecord_name):
    options = tf.io.TFRecordOptions(compression_type='GZIP', compression_level=9)
    with tf.io.TFRecordWriter(tfrecord_name, options=options) as file_writer:
        for i, row in tqdm(chunk.iterrows()):
            record_bytes = encode_row(row)
            file_writer.write(record_bytes)
            del record_bytes
        file_writer.close()

In [9]:
row = train_df.iloc[0]
coordinates = load_relevant_data_subset(f'/kaggle/input/asl-signs/{row.path}')
coordinates_encoded = coordinates.tobytes()
participant_id = int(row.participant_id)
sequence_id = int(row.sequence_id)
sign = int(LABEL_DICT[row.sign])

In [10]:

record_bytes = tf.train.Example(features=tf.train.Features(feature={
            'coordinates': tf.train.Feature(bytes_list=tf.train.BytesList(value=[coordinates_encoded])),
            'participant_id': tf.train.Feature(int64_list=tf.train.Int64List(value=[participant_id])),
            'sequence_id':tf.train.Feature(int64_list=tf.train.Int64List(value=[sequence_id])),
            'sign':tf.train.Feature(int64_list=tf.train.Int64List(value=[sign])),
            }))

In [11]:
N_FILES = len(train_df)
CHUNK_SIZE = 512
N_PART = 1
FOLD = 4
part = 0

class CFG:
    seed = 42
    n_splits = 4

In [12]:
train_folds = train_df.copy()
train_folds['fold']=-1

num_bins = 5

# train_folds = train_folds.sample(frac=1, random_state=CFG.seed).reset_index(drop=True)
# gkfold = StratifiedGroupKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed) 
# print(f'{CFG.n_splits}fold training', len(train_folds), 'samples')
# for fold_idx, (train_idx, valid_idx) in enumerate(gkfold.split(train_folds, y=train_folds['sign'].values, groups=train_folds.participant_id)):
#     train_folds.loc[valid_idx,'fold'] = fold_idx
#     print(f'fold{fold_idx}:', 'train', len(train_idx), 'valid', len(valid_idx))
kfold = KFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed) 
print(f'{CFG.n_splits}fold training', len(train_folds), 'samples')
for fold_idx, (train_idx, valid_idx) in enumerate(kfold.split(train_folds)):
    train_folds.loc[valid_idx,'fold'] = fold_idx
    print(f'fold{fold_idx}:', 'train', len(train_idx), 'valid', len(valid_idx))
    
assert not (train_folds['fold']==-1).sum()
assert len(np.unique(train_folds['fold']))==CFG.n_splits
train_folds.head()

4fold training 94477 samples
fold0: train 70857 valid 23620
fold1: train 70858 valid 23619
fold2: train 70858 valid 23619
fold3: train 70858 valid 23619


,path,participant_id,sequence_id,sign,fold
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow,1
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait,2
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud,3
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird,2
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie,1


In [13]:
%%time

import json
import os

DATASET_NAME = f'ISLR-{CFG.n_splits}fold-randsplit'
### Create Kaggle Dataset if not exists 
# DATASET_NAME='nearestls'

!rm -rf /tmp/{DATASET_NAME}

os.makedirs(f'/tmp/{DATASET_NAME}', exist_ok=True)

with open('/kaggle/input/kaggleapi/kaggle.json') as f:
    kaggle_creds = json.load(f)
    
os.environ['KAGGLE_USERNAME'] = kaggle_creds['username']
os.environ['KAGGLE_KEY'] = kaggle_creds['key']

!kaggle datasets init -p /tmp/{DATASET_NAME}

with open(f'/tmp/{DATASET_NAME}/dataset-metadata.json') as f:
    dataset_meta = json.load(f)

dataset_meta['id'] = f'hoyso48/{DATASET_NAME}'
dataset_meta['title'] = DATASET_NAME
with open(f'/tmp/{DATASET_NAME}/dataset-metadata.json', "w") as outfile:
    json.dump(dataset_meta, outfile)
print(dataset_meta)

!cp /tmp/{DATASET_NAME}/dataset-metadata.json /tmp/{DATASET_NAME}/meta.json
!ls /tmp/{DATASET_NAME}

!kaggle datasets create -p /tmp/{DATASET_NAME} --public

Data package template written to: /tmp/ISLR-4fold-randsplit/dataset-metadata.json
{'title': 'ISLR-4fold-randsplit', 'id': 'hoyso48/ISLR-4fold-randsplit', 'licenses': [{'name': 'CC0-1.0'}]}
dataset-metadata.json  meta.json
Starting upload for file meta.json
100%|███████████████████████████████████████████| 106/106 [00:02<00:00, 44.6B/s]
Upload successful: meta.json (106B)
Your public Dataset is being created. Please check progress at https://www.kaggle.com/datasets/hoyso48/ISLR-4fold-randsplit
CPU times: user 149 ms, sys: 79.2 ms, total: 228 ms
Wall time: 11.8 s


In [14]:
# Put every image in a seperate TFRecord file
# Make Pairs of Views as input to the model


def split_dataframe(df, chunk_size = 10000): 
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

for fold in range(CFG.n_splits):#[FOLD]:#range(CFG.n_splits):
    rows = train_folds[train_folds['fold']==fold]
    chunks = split_dataframe(rows, CHUNK_SIZE)
    part_size = len(chunks)//N_PART
    last = (part+1)*part_size if part != N_PART - 1 else len(chunks)+1
    chunks = chunks[part*part_size:last]
    
    N = [len(x) for x in chunks]
    _ = Parallel(n_jobs=cpu_count())(
        delayed(process_chunk)(x, f'/tmp/{DATASET_NAME}/fold{fold}-{i}-{n}.tfrecords')
        for i,(x,n) in enumerate(zip(chunks,N))
    )

487it [00:21, 27.44it/s]
512it [00:22, 22.36it/s]
497it [00:23, 22.84it/s]
18it [00:00, 23.22it/s]
512it [00:22, 22.39it/s]
512it [00:22, 22.91it/s]
512it [00:22, 22.82it/s]
512it [00:22, 22.90it/s]
512it [00:21, 23.88it/s]
512it [00:21, 24.32it/s]
512it [00:22, 22.35it/s]
512it [00:22, 22.81it/s]
512it [00:22, 22.47it/s]
512it [00:23, 22.12it/s]
512it [00:22, 22.88it/s]
512it [00:23, 22.11it/s]
512it [00:23, 21.59it/s]
512it [00:24, 20.97it/s]
10it [00:00, 29.51it/s]
512it [00:25, 19.92it/s]
512it [00:22, 22.51it/s]
512it [00:20, 24.62it/s]
470it [00:19, 27.19it/s]
512it [00:21, 23.94it/s]
512it [00:21, 23.86it/s]
421it [00:18, 23.01it/s]
69it [00:02, 26.46it/s]
28it [00:01, 18.66it/s]
512it [00:20, 24.54it/s]
512it [00:21, 24.08it/s]
512it [00:21, 24.19it/s]
512it [00:22, 23.08it/s]
512it [00:20, 24.62it/s]
512it [00:20, 24.59it/s]
512it [00:20, 24.82it/s]
512it [00:23, 22.24it/s]
512it [00:20, 25.22it/s]
512it [00:21, 23.84it/s]
87it [00:03, 21.99it/s]
512it [00:21, 24.23it/s]
512it

In [15]:
from datetime import datetime
version_name = datetime.now().strftime("%Y%m%d-%H%M%S")
print(version_name)

20230323-162138


In [16]:
!kaggle datasets version -m {version_name} -p /tmp/{DATASET_NAME} -r zip

Starting upload for file fold1-21-512.tfrecords
100%|████████████████████████████████████████| 111M/111M [00:07<00:00, 15.1MB/s]
Upload successful: fold1-21-512.tfrecords (111MB)
Starting upload for file fold3-14-512.tfrecords
100%|████████████████████████████████████████| 103M/103M [00:06<00:00, 17.8MB/s]
Upload successful: fold3-14-512.tfrecords (103MB)
Starting upload for file fold1-30-512.tfrecords
100%|████████████████████████████████████████| 111M/111M [00:06<00:00, 17.4MB/s]
Upload successful: fold1-30-512.tfrecords (111MB)
Starting upload for file fold1-46-67.tfrecords
100%|██████████████████████████████████████| 18.0M/18.0M [00:03<00:00, 5.97MB/s]
Upload successful: fold1-46-67.tfrecords (18MB)
Starting upload for file fold1-28-512.tfrecords
100%|████████████████████████████████████████| 112M/112M [00:06<00:00, 17.7MB/s]
Upload successful: fold1-28-512.tfrecords (112MB)
Starting upload for file fold2-43-512.tfrecords
100%|████████████████████████████████████████| 105M/105M [00

 53%|████████████████████▍                  | 55.8M/106M [00:03<00:02, 22.3MB/s]

100%|████████████████████████████████████████| 106M/106M [00:06<00:00, 17.5MB/s]
Upload successful: fold3-36-512.tfrecords (106MB)
Starting upload for file fold2-39-512.tfrecords
100%|████████████████████████████████████████| 101M/101M [00:06<00:00, 17.2MB/s]
Upload successful: fold2-39-512.tfrecords (101MB)
Starting upload for file fold2-40-512.tfrecords
 99%|█████████████████████████████████████▊| 99.0M/99.6M [00:04<00:00, 33.4MB/s]

100%|██████████████████████████████████████| 99.6M/99.6M [00:05<00:00, 17.4MB/s]
Upload successful: fold2-40-512.tfrecords (100MB)
Starting upload for file fold2-3-512.tfrecords
  0%|                                                | 0.00/101M [00:00<?, ?B/s]

100%|████████████████████████████████████████| 101M/101M [00:06<00:00, 16.5MB/s]
Upload successful: fold2-3-512.tfrecords (101MB)
Starting upload for file fold3-15-512.tfrecords
100%|████████████████████████████████████████| 102M/102M [00:06<00:00, 17.6MB/s]
Upload successful: fold3-15-512.tfrecords (102MB)
Starting upload for file fold2-36-512.tfrecords
100%|████████████████████████████████████████| 111M/111M [00:06<00:00, 18.7MB/s]
Upload successful: fold2-36-512.tfrecords (111MB)
Starting upload for file fold3-8-512.tfrecords
100%|██████████████████████████████████████| 99.0M/99.0M [00:07<00:00, 14.7MB/s]
Upload successful: fold3-8-512.tfrecords (99MB)
Starting upload for file fold3-11-512.tfrecords
100%|████████████████████████████████████████| 109M/109M [00:06<00:00, 18.1MB/s]
Upload successful: fold3-11-512.tfrecords (109MB)
Starting upload for file fold0-35-512.tfrecords
100%|████████████████████████████████████████| 116M/116M [00:07<00:00, 17.0MB/s]
Upload successful: fold0-35-

In [17]:
from kaggle_datasets import KaggleDatasets
print(KaggleDatasets().get_gcs_path('islr-5fold'))
print(KaggleDatasets().get_gcs_path('islr-5fold-randsplit'))
# print(KaggleDatasets().get_gcs_path('islr-5fold-fold0'))
# print(KaggleDatasets().get_gcs_path('islr-5fold-fold1'))
# print(KaggleDatasets().get_gcs_path('islr-5fold-fold2'))
# print(KaggleDatasets().get_gcs_path('islr-5fold-fold3'))
# print(KaggleDatasets().get_gcs_path('islr-5fold-fold4'))

gs://kds-567b429b7f8165983d388349911e33336b75f39849d653e23ab5b402
gs://kds-91f2c540db6c6411ec085e1652399d60d4b0e6c5bbb5e8186519f38c
